In [2]:
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken

In [3]:
import sys
import os

class SuppressStdout:
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings

loader = PyPDFLoader("/root/PediatricDrugDoses.pdf")
pages = loader.load_and_split()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/root/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/root/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/root/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`

In [5]:
from langchain.vectorstores import Chroma
with SuppressStdout():
    vectordb = Chroma.from_documents(documents=pages, embedding=embeddings,)

In [33]:
from langchain_community.llms.ollama import Ollama

llm = Ollama(
    model="llama3",
    temperature=0,
    top_k=10,
    top_p=0.1,
)

In [128]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [130]:
from langchain.prompts import PromptTemplate

template = """You are healthcare Assistant who suggests pediatric drug dosage.  
Human will give you the symptoms that he/she is experiencing.
You will be provided with a context from our database that contains information relevant to the symptoms described by the Human.
Also analyze the context if the mentioned symptoms are there in it. If not, ask a relevant questions like "Tell me more about the symptoms"
If you are not sure that it is the right dosage, ask questions related to the context to be 100 percent sure.
If you are still not sure after asking all the questions, simply respond with "I don't know. Can you ask another question".
If questions are asked where there is no relevant context available, simply respond with "I don't know. Please ask a question relevant to the documents"
You'll also get the chat history, so that you can collect all the symptoms of the user and make a clear decision. Make sure you do not ask the same questions again.
Once you are sure of the dosage, suggest it to the human and do not ask additional questions, ignore the context.
Chat history: {chat_history}

Context: {context}

Human: {question}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"], template=template
)

# Create the custom chain
if llm is not None and vectordb is not None:
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm, 
        retriever=vectordb.as_retriever(search_type='mmr', search_kwargs={'k':2, 'fetch_k': 50, 'lambda_mult': 0.2}), 
        memory=memory,
        chain_type="stuff",
        verbose=True,
        combine_docs_chain_kwargs={'prompt': prompt})
else:
    print("LLM or Vector Database not initialized")

In [ ]:
chain.invoke("Patient is having vommiting")

In [ ]:
# input = "I am having Fever"
# all_messages = ""
# for memid,message in enumerate(memory.chat_memory.messages):
#     if memid%2 != 0:
#         message = message[memid]
#     all_messages += message
    
# chain.invoke(input+"<PREV_MESSAGES>"+all_messages)